In [1]:
!pip install nltk

In [1]:
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd

In [395]:
df_train = pd.read_csv('train_spam.csv')
df_test = pd.read_csv('test_spam.csv')

In [396]:
df_train

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...
...,...,...
16273,spam,if you are interested in binary options tradin...
16274,spam,dirty pictureblyk on aircel thanks you for bei...
16275,ham,or you could do this g on mon 1635465 sep 1635...
16276,ham,insta reels par 80 गंद bhara pada hai 👀 kuch b...


Сделаем целевую переменную целочисленной

In [397]:
def change(s: str):
  return int(s == 'spam')

In [398]:
df_train['text_type'] = df_train['text_type'].apply(lambda x: change(x))

In [399]:
df_train

,text_type,text
0,0,make sure alex knows his birthday is over in f...
1,0,a resume for john lavorato thanks vince i will...
2,1,plzz visit my website moviesgodml to get all m...
3,1,urgent your mobile number has been awarded wit...
4,0,overview of hr associates analyst project per ...
...,...,...
16273,1,if you are interested in binary options tradin...
16274,1,dirty pictureblyk on aircel thanks you for bei...
16275,0,or you could do this g on mon 1635465 sep 1635...
16276,0,insta reels par 80 गंद bhara pada hai 👀 kuch b...


In [400]:
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords
from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [401]:
noise = noise = stopwords.words('english') + list(punctuation)

Разделим текст на токены, удалим стоп-слова и пунктуацию, а также застеммим слова

In [402]:
def text_to_vector(s: str, type: int):
  text_tokenized = [w for w in word_tokenize(s) if w.isalpha() and w not in noise]
  stemmer = SnowballStemmer('english')
  text_tokens = [stemmer.stem(w) for w in text_tokenized]
  if type == 2:
    return text_tokens
  text = ' '.join(text_tokens)
  return text

In [428]:
df = df_train.copy()
# df_train['text_tokenized'] = df['text'].apply(lambda x: text_to_vector(x, 1))
df_train['text_tokens'] = df['text'].apply(lambda x: text_to_vector(x, 2))
df_test['text_tokens'] = df_test['text'].apply(lambda x: text_to_vector(x, 2))
df_train

,text_type,text,text_tokens
0,0,make sure alex knows his birthday is over in f...,"[make, sure, alex, know, birthday, fifteen, mi..."
1,0,a resume for john lavorato thanks vince i will...,"[resum, john, lavorato, thank, vinc, get, move..."
2,1,plzz visit my website moviesgodml to get all m...,"[plzz, visit, websit, moviesgodml, get, movi, ..."
3,1,urgent your mobile number has been awarded wit...,"[urgent, mobil, number, award, prize, guarante..."
4,0,overview of hr associates analyst project per ...,"[overview, hr, associ, analyst, project, per, ..."
...,...,...,...
16273,1,if you are interested in binary options tradin...,"[interest, binari, option, trade, may, continu..."
16274,1,dirty pictureblyk on aircel thanks you for bei...,"[dirti, pictureblyk, aircel, thank, valu, memb..."
16275,0,or you could do this g on mon 1635465 sep 1635...,"[could, g, mon, sep, david, ree, wrote, mon, s..."
16276,0,insta reels par 80 गंद bhara pada hai 👀 kuch b...,"[insta, reel, par, bhara, pada, hai, kuch, bhi..."


In [429]:
df_test

,text,text_tokens
0,j jim whitehead ejw cse ucsc edu writes j you ...,"[j, jim, whitehead, ejw, cse, ucsc, edu, write..."
1,original message from bitbitch magnesium net p...,"[origin, messag, bitbitch, magnesium, net, peo..."
2,java for managers vince durasoft who just taug...,"[java, manag, vinc, durasoft, taught, java, cl..."
3,there is a youtuber name saiman says,"[youtub, name, saiman, say]"
4,underpriced issue with high return on equity t...,"[underpr, issu, high, return, equiti, oil, gas..."
...,...,...
4065,husband to wifetum meri zindagi hoorwifeor kya...,"[husband, wifetum, meri, zindagi, hoorwifeor, ..."
4066,baylor enron case study cindy yes i shall co a...,"[baylor, enron, case, studi, cindi, yes, shall..."
4067,boring as compared to tp,"[bore, compar, tp]"
4068,hellogorgeous hows u my fone was on charge lst...,"[hellogorg, how, u, fone, charg, lst, nitw, we..."


In [27]:
!pip install gensim

Не будем проверять bag-of-words и tf-idf, сразу возьмем word2vec

In [406]:
import gensim
from gensim.models import Word2Vec

In [430]:
vec_size = 200

Не совсем честно использовать word2vec, обученный на тестовых данных, но раз уж они нам доступны, будем хитрить

In [441]:
texts = pd.DataFrame(df_train[['text_tokens']].values.tolist() + df_test[['text_tokens']].values.tolist(), columns = ['text_tokens'])
# texts = pd.concat([texts, df_test['text_tokens']], ignore_index=True)
texts

,text_tokens
0,"[make, sure, alex, know, birthday, fifteen, mi..."
1,"[resum, john, lavorato, thank, vinc, get, move..."
2,"[plzz, visit, websit, moviesgodml, get, movi, ..."
3,"[urgent, mobil, number, award, prize, guarante..."
4,"[overview, hr, associ, analyst, project, per, ..."
...,...
20343,"[husband, wifetum, meri, zindagi, hoorwifeor, ..."
20344,"[baylor, enron, case, studi, cindi, yes, shall..."
20345,"[bore, compar, tp]"
20346,"[hellogorg, how, u, fone, charg, lst, nitw, we..."


In [442]:
model = Word2Vec(texts['text_tokens'], vector_size=vec_size, window=5, min_count=1, sg=0)

In [443]:
def get_embedding(s: str):
  vec = []
  for word in s:
    if word in model.wv:
      vec.append(model.wv[word])
    if len(vec) == 0:
      return np.zeros(model.vector_size)
    return np.mean(vec, axis = 0)

Получаем эмбеддинги текстов как среднее эмбеддингов всех слов

In [444]:
df_train['text_vec'] = df_train['text_tokens'].apply(get_embedding)
df_train

,text_type,text,text_tokens,text_vec
0,0,make sure alex knows his birthday is over in f...,"[make, sure, alex, know, birthday, fifteen, mi...","[-0.3266853, -0.25703755, 0.49778154, 0.002815..."
1,0,a resume for john lavorato thanks vince i will...,"[resum, john, lavorato, thank, vinc, get, move...","[0.093269765, -0.33026692, -0.90982395, 0.4333..."
2,1,plzz visit my website moviesgodml to get all m...,"[plzz, visit, websit, moviesgodml, get, movi, ...","[0.0041364767, -0.018255213, 0.00677969, 0.035..."
3,1,urgent your mobile number has been awarded wit...,"[urgent, mobil, number, award, prize, guarante...","[-0.00974447, -0.30572337, -0.1625484, -0.1469..."
4,0,overview of hr associates analyst project per ...,"[overview, hr, associ, analyst, project, per, ...","[-0.041077726, -0.047182262, -0.0017376217, 0...."
...,...,...,...,...
16273,1,if you are interested in binary options tradin...,"[interest, binari, option, trade, may, continu...","[0.08109808, -0.5035061, -0.4502283, -0.041978..."
16274,1,dirty pictureblyk on aircel thanks you for bei...,"[dirti, pictureblyk, aircel, thank, valu, memb...","[0.060444493, -0.07033851, 0.04162678, 0.10058..."
16275,0,or you could do this g on mon 1635465 sep 1635...,"[could, g, mon, sep, david, ree, wrote, mon, s...","[0.08103398, -0.59803796, -0.0012330165, -0.21..."
16276,0,insta reels par 80 गंद bhara pada hai 👀 kuch b...,"[insta, reel, par, bhara, pada, hai, kuch, bhi...","[0.023600062, -0.026595606, 0.0165888, 0.03284..."


In [457]:
df_final = df_train[['text_vec', 'text_type']]
df_final.dropna()
df_final

,text_vec,text_type
0,"[-0.3266853, -0.25703755, 0.49778154, 0.002815...",0
1,"[0.093269765, -0.33026692, -0.90982395, 0.4333...",0
2,"[0.0041364767, -0.018255213, 0.00677969, 0.035...",1
3,"[-0.00974447, -0.30572337, -0.1625484, -0.1469...",1
4,"[-0.041077726, -0.047182262, -0.0017376217, 0....",0
...,...,...
16273,"[0.08109808, -0.5035061, -0.4502283, -0.041978...",1
16274,"[0.060444493, -0.07033851, 0.04162678, 0.10058...",1
16275,"[0.08103398, -0.59803796, -0.0012330165, -0.21...",0
16276,"[0.023600062, -0.026595606, 0.0165888, 0.03284...",0


В некоторых вхождениях вместо вектора эмбеддинга встречается None, найдем их и удалим

In [458]:
new_vec = []
for i in range(len(df_final)):
  vector = df_final.iloc[i]['text_vec']
  new_vec.append(vector)

In [459]:
none_ind = [ind for ind, x in enumerate(new_vec) if x is None]

In [460]:
df_final.drop(none_ind, inplace = True)
df_final = df_final.reset_index(drop = True)

<ipython-input-460-13c82ea1bb84>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.drop(none_ind, inplace = True)


In [461]:
df_final

,text_vec,text_type
0,"[-0.3266853, -0.25703755, 0.49778154, 0.002815...",0
1,"[0.093269765, -0.33026692, -0.90982395, 0.4333...",0
2,"[0.0041364767, -0.018255213, 0.00677969, 0.035...",1
3,"[-0.00974447, -0.30572337, -0.1625484, -0.1469...",1
4,"[-0.041077726, -0.047182262, -0.0017376217, 0....",0
...,...,...
16222,"[0.08109808, -0.5035061, -0.4502283, -0.041978...",1
16223,"[0.060444493, -0.07033851, 0.04162678, 0.10058...",1
16224,"[0.08103398, -0.59803796, -0.0012330165, -0.21...",0
16225,"[0.023600062, -0.026595606, 0.0165888, 0.03284...",0


In [462]:
cols = [f'text_vec_{i}' for i in range(vec_size)]
df_final[cols] = pd.DataFrame(df_final['text_vec'].tolist())
# df_target = df_final['text_type'].copy(deep = True)
# df_final1 = pd.concat([df_vec, df_target], axis = 1)

df_final
# df_final['text_type'].copy()

<ipython-input-462-92a8b392c98a>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[cols] = pd.DataFrame(df_final['text_vec'].tolist())
<ipython-input-462-92a8b392c98a>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[cols] = pd.DataFrame(df_final['text_vec'].tolist())
<ipython-input-462-92a8b392c98a>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inst

,text_vec,text_type,text_vec_0,text_vec_1,text_vec_2,text_vec_3,text_vec_4,text_vec_5,text_vec_6,text_vec_7,...,text_vec_190,text_vec_191,text_vec_192,text_vec_193,text_vec_194,text_vec_195,text_vec_196,text_vec_197,text_vec_198,text_vec_199
0,"[-0.3266853, -0.25703755, 0.49778154, 0.002815...",0,-0.326685,-0.257038,0.497782,0.002815,0.710172,0.119826,-1.249085,0.773005,...,0.345734,-0.543234,0.378910,0.017017,0.808543,0.043182,0.860116,-0.771000,-0.883956,-0.044132
1,"[0.093269765, -0.33026692, -0.90982395, 0.4333...",0,0.093270,-0.330267,-0.909824,0.433373,0.608252,-0.327772,0.949404,2.341581,...,0.743486,0.216215,-0.287586,-1.584474,0.333398,0.216706,-0.064091,0.106825,-0.502335,-0.645614
2,"[0.0041364767, -0.018255213, 0.00677969, 0.035...",1,0.004136,-0.018255,0.006780,0.035710,0.040755,-0.038092,-0.020173,0.067252,...,0.028851,-0.022318,-0.028237,-0.029931,0.026922,-0.012806,0.009302,-0.050493,0.003220,-0.010083
3,"[-0.00974447, -0.30572337, -0.1625484, -0.1469...",1,-0.009744,-0.305723,-0.162548,-0.146980,0.480372,-0.181383,-0.256530,0.542971,...,0.477168,-0.310388,-0.281209,-0.580615,0.119691,0.091480,0.356655,-0.596363,0.068322,-0.483876
4,"[-0.041077726, -0.047182262, -0.0017376217, 0....",0,-0.041078,-0.047182,-0.001738,0.045062,0.183502,-0.086471,-0.127833,0.302020,...,0.018759,-0.022849,-0.109938,-0.160662,0.097261,-0.041745,0.011954,-0.120421,-0.016184,-0.046872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16222,"[0.08109808, -0.5035061, -0.4502283, -0.041978...",1,0.081098,-0.503506,-0.450228,-0.041979,0.691850,-0.032272,-0.452953,1.320112,...,0.175441,0.270193,-0.111294,-0.923211,1.011635,0.448082,0.544518,-0.364149,-0.650219,-0.356181
16223,"[0.060444493, -0.07033851, 0.04162678, 0.10058...",1,0.060444,-0.070339,0.041627,0.100589,0.104105,-0.178852,-0.114900,0.218676,...,0.134358,-0.087546,-0.036591,-0.132268,0.125396,0.039549,0.142076,-0.247767,-0.038727,-0.070996
16224,"[0.08103398, -0.59803796, -0.0012330165, -0.21...",0,0.081034,-0.598038,-0.001233,-0.216100,0.025643,-0.067247,-0.418138,1.124433,...,0.451088,-0.152325,-0.178218,-0.482546,0.597436,0.503234,0.679848,-0.325717,-1.026731,-0.233681
16225,"[0.023600062, -0.026595606, 0.0165888, 0.03284...",0,0.023600,-0.026596,0.016589,0.032843,0.012881,-0.051636,-0.041942,0.072694,...,0.033625,-0.034575,-0.033615,-0.037476,0.034172,-0.003672,0.047725,-0.084957,0.000658,-0.014108


In [463]:
df_final = df_final.drop('text_vec', axis=1)

Получаем финальный датасет, на котором можно обучать модель

In [464]:
cols = df_final.columns.tolist()
cols = cols[1:] + [cols[0]]
df_final = df_final.reindex(columns=cols)
df_final

,text_vec_0,text_vec_1,text_vec_2,text_vec_3,text_vec_4,text_vec_5,text_vec_6,text_vec_7,text_vec_8,text_vec_9,...,text_vec_191,text_vec_192,text_vec_193,text_vec_194,text_vec_195,text_vec_196,text_vec_197,text_vec_198,text_vec_199,text_type
0,-0.326685,-0.257038,0.497782,0.002815,0.710172,0.119826,-1.249085,0.773005,-0.336148,0.568484,...,-0.543234,0.378910,0.017017,0.808543,0.043182,0.860116,-0.771000,-0.883956,-0.044132,0
1,0.093270,-0.330267,-0.909824,0.433373,0.608252,-0.327772,0.949404,2.341581,0.778954,-0.201247,...,0.216215,-0.287586,-1.584474,0.333398,0.216706,-0.064091,0.106825,-0.502335,-0.645614,0
2,0.004136,-0.018255,0.006780,0.035710,0.040755,-0.038092,-0.020173,0.067252,-0.015419,0.032298,...,-0.022318,-0.028237,-0.029931,0.026922,-0.012806,0.009302,-0.050493,0.003220,-0.010083,1
3,-0.009744,-0.305723,-0.162548,-0.146980,0.480372,-0.181383,-0.256530,0.542971,0.170475,0.138428,...,-0.310388,-0.281209,-0.580615,0.119691,0.091480,0.356655,-0.596363,0.068322,-0.483876,1
4,-0.041078,-0.047182,-0.001738,0.045062,0.183502,-0.086471,-0.127833,0.302020,-0.054923,0.137405,...,-0.022849,-0.109938,-0.160662,0.097261,-0.041745,0.011954,-0.120421,-0.016184,-0.046872,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16222,0.081098,-0.503506,-0.450228,-0.041979,0.691850,-0.032272,-0.452953,1.320112,0.140243,1.206402,...,0.270193,-0.111294,-0.923211,1.011635,0.448082,0.544518,-0.364149,-0.650219,-0.356181,1
16223,0.060444,-0.070339,0.041627,0.100589,0.104105,-0.178852,-0.114900,0.218676,-0.073783,0.154142,...,-0.087546,-0.036591,-0.132268,0.125396,0.039549,0.142076,-0.247767,-0.038727,-0.070996,1
16224,0.081034,-0.598038,-0.001233,-0.216100,0.025643,-0.067247,-0.418138,1.124433,-0.062490,0.629152,...,-0.152325,-0.178218,-0.482546,0.597436,0.503234,0.679848,-0.325717,-1.026731,-0.233681,0
16225,0.023600,-0.026596,0.016589,0.032843,0.012881,-0.051636,-0.041942,0.072694,-0.012176,0.041753,...,-0.034575,-0.033615,-0.037476,0.034172,-0.003672,0.047725,-0.084957,0.000658,-0.014108,0


In [454]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

Разбиваем данные на трейн и тест

In [465]:
X_train, X_test, y_train, y_test = train_test_split(df_final[cols[:-1]], df_final['text_type'], train_size = 0.85, random_state=18)

In [321]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# X_train = ss.fit_transform(X_train)
# X_test = ss.transform(X_test)

In [466]:
X_train.head()

,text_vec_0,text_vec_1,text_vec_2,text_vec_3,text_vec_4,text_vec_5,text_vec_6,text_vec_7,text_vec_8,text_vec_9,...,text_vec_190,text_vec_191,text_vec_192,text_vec_193,text_vec_194,text_vec_195,text_vec_196,text_vec_197,text_vec_198,text_vec_199
1838,-0.005760,0.001915,-0.071975,0.181099,0.308777,-0.160998,0.014446,0.643578,-0.119804,0.073140,...,0.128792,-0.037020,-0.279317,-0.270880,-0.059255,-0.109656,-0.094360,-0.045613,0.062989,-0.168907
3437,0.008370,-0.001349,0.010456,-0.005885,0.014446,0.005702,-0.017294,-0.005102,-0.008562,0.014460,...,0.006219,-0.012351,-0.014959,0.005757,0.007640,-0.005271,0.007217,-0.007717,-0.000504,-0.010440
7900,0.002623,-0.002424,-0.013148,-0.002164,0.008903,-0.001093,-0.001921,0.013898,0.005044,-0.002207,...,-0.001005,0.004879,-0.001792,-0.003261,0.002887,-0.000996,0.000145,-0.007411,0.007230,-0.007246
5105,-0.497179,0.030670,0.429382,0.989053,0.108312,-0.165735,-0.431906,1.309138,-0.704890,1.076668,...,-0.591785,0.245941,-0.830048,-0.661399,0.296705,-0.501187,-0.195026,-0.307969,-0.267374,0.299372
3227,0.093182,-0.029319,0.523690,0.033217,-0.028576,-0.381636,-0.498628,-0.208709,0.197806,0.413194,...,0.858211,0.162697,1.171617,-0.819735,-0.251041,0.532367,0.764744,-0.473871,-0.743131,-0.388943


Попробуем логистическую регрессию, а также случайный лес и градиентный бустинг

In [474]:
clf = LogisticRegression(max_iter=500, random_state=42, class_weight = 'balanced')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))
print(f'roc-auc score = {roc_auc_score(y_test, pred)}')

              precision    recall  f1-score   support

           0       0.88      0.64      0.74      1686
           1       0.50      0.80      0.61       749

    accuracy                           0.69      2435
   macro avg       0.69      0.72      0.68      2435
weighted avg       0.76      0.69      0.70      2435

roc-auc score = 0.7202244352691687


In [469]:
from sklearn.ensemble import RandomForestClassifier

In [475]:
RFC500 = RandomForestClassifier(n_estimators = 500, criterion = 'gini', max_depth = 3, random_state=10, class_weight = 'balanced', max_features = 'sqrt')
RFC500.fit(X_train, y_train)
pred = RFC500.predict(X_test)
print(classification_report(y_test, pred))
print(f'roc-auc score = {roc_auc_score(y_test, pred)}')

              precision    recall  f1-score   support

           0       0.86      0.69      0.77      1686
           1       0.52      0.74      0.61       749

    accuracy                           0.71      2435
   macro avg       0.69      0.72      0.69      2435
weighted avg       0.75      0.71      0.72      2435

roc-auc score = 0.7176179548215336


In [267]:
from sklearn.ensemble import GradientBoostingClassifier

In [476]:
GBC200 = GradientBoostingClassifier(n_estimators = 200, max_depth = 3, subsample = 0.4, max_features = 'sqrt', random_state=10)
GBC200.fit(X_train, y_train)
pred = GBC200.predict(X_test)
print(classification_report(y_test, pred))
print(f'roc-auc score = {roc_auc_score(y_test, pred)}')

              precision    recall  f1-score   support

           0       0.81      0.90      0.85      1686
           1       0.70      0.54      0.61       749

    accuracy                           0.79      2435
   macro avg       0.76      0.72      0.73      2435
weighted avg       0.78      0.79      0.78      2435

roc-auc score = 0.7169785098993201


Как мы видим из отчетов, все работает примерно одинаково, но грубая сила решает, поэтому берем последнюю конфигурацию

In [477]:
GBC500 = GradientBoostingClassifier(n_estimators = 500, max_depth = 3, subsample = 1, max_features = 'sqrt', random_state=18, warm_start = True)
GBC500.fit(X_train, y_train)
pred = GBC500.predict(X_test)
print(classification_report(y_test, pred))
print(f'roc-auc score = {roc_auc_score(y_test, pred)}')

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1686
           1       0.71      0.60      0.65       749

    accuracy                           0.80      2435
   macro avg       0.77      0.74      0.75      2435
weighted avg       0.79      0.80      0.80      2435

roc-auc score = 0.7449438317915386


Обучим финальную модель на всем тренировочном датасете

In [480]:
X_train_final = df_final[cols[:-1]]
y_train_final = df_final['text_type']
X_train_final

,text_vec_0,text_vec_1,text_vec_2,text_vec_3,text_vec_4,text_vec_5,text_vec_6,text_vec_7,text_vec_8,text_vec_9,...,text_vec_190,text_vec_191,text_vec_192,text_vec_193,text_vec_194,text_vec_195,text_vec_196,text_vec_197,text_vec_198,text_vec_199
0,-0.326685,-0.257038,0.497782,0.002815,0.710172,0.119826,-1.249085,0.773005,-0.336148,0.568484,...,0.345734,-0.543234,0.378910,0.017017,0.808543,0.043182,0.860116,-0.771000,-0.883956,-0.044132
1,0.093270,-0.330267,-0.909824,0.433373,0.608252,-0.327772,0.949404,2.341581,0.778954,-0.201247,...,0.743486,0.216215,-0.287586,-1.584474,0.333398,0.216706,-0.064091,0.106825,-0.502335,-0.645614
2,0.004136,-0.018255,0.006780,0.035710,0.040755,-0.038092,-0.020173,0.067252,-0.015419,0.032298,...,0.028851,-0.022318,-0.028237,-0.029931,0.026922,-0.012806,0.009302,-0.050493,0.003220,-0.010083
3,-0.009744,-0.305723,-0.162548,-0.146980,0.480372,-0.181383,-0.256530,0.542971,0.170475,0.138428,...,0.477168,-0.310388,-0.281209,-0.580615,0.119691,0.091480,0.356655,-0.596363,0.068322,-0.483876
4,-0.041078,-0.047182,-0.001738,0.045062,0.183502,-0.086471,-0.127833,0.302020,-0.054923,0.137405,...,0.018759,-0.022849,-0.109938,-0.160662,0.097261,-0.041745,0.011954,-0.120421,-0.016184,-0.046872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16222,0.081098,-0.503506,-0.450228,-0.041979,0.691850,-0.032272,-0.452953,1.320112,0.140243,1.206402,...,0.175441,0.270193,-0.111294,-0.923211,1.011635,0.448082,0.544518,-0.364149,-0.650219,-0.356181
16223,0.060444,-0.070339,0.041627,0.100589,0.104105,-0.178852,-0.114900,0.218676,-0.073783,0.154142,...,0.134358,-0.087546,-0.036591,-0.132268,0.125396,0.039549,0.142076,-0.247767,-0.038727,-0.070996
16224,0.081034,-0.598038,-0.001233,-0.216100,0.025643,-0.067247,-0.418138,1.124433,-0.062490,0.629152,...,0.451088,-0.152325,-0.178218,-0.482546,0.597436,0.503234,0.679848,-0.325717,-1.026731,-0.233681
16225,0.023600,-0.026596,0.016589,0.032843,0.012881,-0.051636,-0.041942,0.072694,-0.012176,0.041753,...,0.033625,-0.034575,-0.033615,-0.037476,0.034172,-0.003672,0.047725,-0.084957,0.000658,-0.014108


In [481]:
GBC_model = GradientBoostingClassifier(n_estimators = 500, max_depth = 3, subsample = 1, max_features = 'sqrt', random_state=18, warm_start = True)
GBC_model.fit(X_train_final, y_train_final)

GradientBoostingClassifier(max_features='sqrt', n_estimators=500,
                           random_state=18, subsample=1, warm_start=True)

Вернемся к тестовым данным и подготовим их так же, как тренировочные

In [482]:
df_test

,text,text_tokens
0,j jim whitehead ejw cse ucsc edu writes j you ...,"[j, jim, whitehead, ejw, cse, ucsc, edu, write..."
1,original message from bitbitch magnesium net p...,"[origin, messag, bitbitch, magnesium, net, peo..."
2,java for managers vince durasoft who just taug...,"[java, manag, vinc, durasoft, taught, java, cl..."
3,there is a youtuber name saiman says,"[youtub, name, saiman, say]"
4,underpriced issue with high return on equity t...,"[underpr, issu, high, return, equiti, oil, gas..."
...,...,...
4065,husband to wifetum meri zindagi hoorwifeor kya...,"[husband, wifetum, meri, zindagi, hoorwifeor, ..."
4066,baylor enron case study cindy yes i shall co a...,"[baylor, enron, case, studi, cindi, yes, shall..."
4067,boring as compared to tp,"[bore, compar, tp]"
4068,hellogorgeous hows u my fone was on charge lst...,"[hellogorg, how, u, fone, charg, lst, nitw, we..."


In [483]:
df_test['text_vec'] = df_test['text_tokens'].apply(get_embedding)
df_test

,text,text_tokens,text_vec
0,j jim whitehead ejw cse ucsc edu writes j you ...,"[j, jim, whitehead, ejw, cse, ucsc, edu, write...","[0.92032653, -1.1923071, -1.1894892, 0.8631177..."
1,original message from bitbitch magnesium net p...,"[origin, messag, bitbitch, magnesium, net, peo...","[0.75240606, -0.67271566, -0.76806056, 0.04310..."
2,java for managers vince durasoft who just taug...,"[java, manag, vinc, durasoft, taught, java, cl...","[0.1117042, -0.12201274, 0.012747883, 0.165301..."
3,there is a youtuber name saiman says,"[youtub, name, saiman, say]","[0.1537048, -0.31716663, 0.058771744, -0.02734..."
4,underpriced issue with high return on equity t...,"[underpr, issu, high, return, equiti, oil, gas...","[0.0058211465, -0.007440747, 0.002752937, 0.00..."
...,...,...,...
4065,husband to wifetum meri zindagi hoorwifeor kya...,"[husband, wifetum, meri, zindagi, hoorwifeor, ...","[0.02183394, -0.037809655, 0.18611246, 0.25754..."
4066,baylor enron case study cindy yes i shall co a...,"[baylor, enron, case, studi, cindi, yes, shall...","[0.20528184, -0.14732066, -0.37170982, 0.04815..."
4067,boring as compared to tp,"[bore, compar, tp]","[0.06885123, -0.06677675, 0.06560548, 0.120124..."
4068,hellogorgeous hows u my fone was on charge lst...,"[hellogorg, how, u, fone, charg, lst, nitw, we...","[0.0038270634, 0.00085642765, -0.0018486775, 0..."


Опять убираем none-ы (я так и не понял, откуда они берутся)

In [484]:
def replace_none(a):
  if a is None:
    return np.zeros(vec_size)
  return a

In [485]:
df_test['text_vec'] = df_test['text_vec'].apply(replace_none)
df_test

,text,text_tokens,text_vec
0,j jim whitehead ejw cse ucsc edu writes j you ...,"[j, jim, whitehead, ejw, cse, ucsc, edu, write...","[0.92032653, -1.1923071, -1.1894892, 0.8631177..."
1,original message from bitbitch magnesium net p...,"[origin, messag, bitbitch, magnesium, net, peo...","[0.75240606, -0.67271566, -0.76806056, 0.04310..."
2,java for managers vince durasoft who just taug...,"[java, manag, vinc, durasoft, taught, java, cl...","[0.1117042, -0.12201274, 0.012747883, 0.165301..."
3,there is a youtuber name saiman says,"[youtub, name, saiman, say]","[0.1537048, -0.31716663, 0.058771744, -0.02734..."
4,underpriced issue with high return on equity t...,"[underpr, issu, high, return, equiti, oil, gas...","[0.0058211465, -0.007440747, 0.002752937, 0.00..."
...,...,...,...
4065,husband to wifetum meri zindagi hoorwifeor kya...,"[husband, wifetum, meri, zindagi, hoorwifeor, ...","[0.02183394, -0.037809655, 0.18611246, 0.25754..."
4066,baylor enron case study cindy yes i shall co a...,"[baylor, enron, case, studi, cindi, yes, shall...","[0.20528184, -0.14732066, -0.37170982, 0.04815..."
4067,boring as compared to tp,"[bore, compar, tp]","[0.06885123, -0.06677675, 0.06560548, 0.120124..."
4068,hellogorgeous hows u my fone was on charge lst...,"[hellogorg, how, u, fone, charg, lst, nitw, we...","[0.0038270634, 0.00085642765, -0.0018486775, 0..."


In [486]:
cols = [f'text_vec_{i}' for i in range(vec_size)]
X_test_final = pd.DataFrame(df_test['text_vec'].tolist(), columns = cols)

Получили подготовленные данные, по которым необходимо предсказать "спаммность" текстов

In [488]:
X_test_final

,text_vec_0,text_vec_1,text_vec_2,text_vec_3,text_vec_4,text_vec_5,text_vec_6,text_vec_7,text_vec_8,text_vec_9,...,text_vec_190,text_vec_191,text_vec_192,text_vec_193,text_vec_194,text_vec_195,text_vec_196,text_vec_197,text_vec_198,text_vec_199
0,0.920327,-1.192307,-1.189489,0.863118,0.362340,-0.322319,0.396708,3.545913,1.356599,-1.492316,...,-0.092204,-0.522081,0.255810,-1.731296,-0.936938,0.462508,-0.012752,-0.558212,0.034591,-0.585130
1,0.752406,-0.672716,-0.768061,0.043102,0.289976,-0.364237,0.440302,1.250398,1.007872,-0.935730,...,0.694173,-0.141104,0.027026,-0.836117,0.304009,0.511612,0.741933,-0.207119,-0.128290,-0.871288
2,0.111704,-0.122013,0.012748,0.165301,0.087492,-0.296421,-0.244662,0.448750,-0.127848,0.191884,...,0.086820,-0.192889,-0.147930,-0.116130,0.136389,0.065351,-0.001966,-0.224285,-0.118249,-0.008649
3,0.153705,-0.317167,0.058772,-0.027348,0.242738,-0.320321,-0.385522,0.468175,-0.151592,0.429070,...,0.422361,-0.436011,-0.308105,-0.244540,0.439915,0.041778,0.462544,-0.763596,-0.050312,-0.170065
4,0.005821,-0.007441,0.002753,0.001988,0.003366,-0.000996,0.001622,0.005647,0.002842,-0.007674,...,0.005900,-0.002159,-0.001785,0.000507,0.008138,-0.000885,0.000274,-0.003560,-0.007477,-0.007758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4065,0.021834,-0.037810,0.186112,0.257544,0.142930,-0.313440,-0.179436,0.308415,-0.098701,0.239561,...,0.240605,-0.000027,0.122173,-0.325227,0.022806,0.032854,0.229222,-0.323226,-0.114349,-0.127711
4066,0.205282,-0.147321,-0.371710,0.048152,0.598999,-0.525578,0.066129,1.000919,-0.175500,-0.134209,...,-0.247520,-0.203461,-0.255706,-0.292832,0.338497,-0.031651,-0.063354,-0.244280,0.105271,-0.122771
4067,0.068851,-0.066777,0.065605,0.120124,0.042453,-0.260519,-0.153320,0.153354,-0.026215,0.183334,...,0.231441,-0.041298,0.176222,-0.253349,-0.002835,0.128803,0.133645,-0.256600,-0.126773,-0.115884
4068,0.003827,0.000856,-0.001849,0.003640,0.006764,-0.004143,-0.007480,0.008344,-0.001042,-0.000562,...,0.007761,-0.001405,0.003782,-0.009334,-0.001783,0.003731,-0.003147,-0.001545,0.001059,-0.003017


Момент истины...

In [489]:
final_prediction = GBC_model.predict(X_test_final)

Вернем данные в правильный вид

In [490]:
def change_back(type: int):
  if type == 1:
    return 'spam'
  return 'ham'

In [499]:
df_predicted = pd.DataFrame(final_prediction, columns = ['int_pred'])
df_predicted['score'] = df_predicted['int_pred'].apply(change_back)
df_predicted['text'] = df_test['text'].copy()
df_predicted = df_predicted.drop('int_pred', axis=1)
df_predicted

,score,text
0,ham,j jim whitehead ejw cse ucsc edu writes j you ...
1,ham,original message from bitbitch magnesium net p...
2,ham,java for managers vince durasoft who just taug...
3,ham,there is a youtuber name saiman says
4,ham,underpriced issue with high return on equity t...
...,...,...
4065,ham,husband to wifetum meri zindagi hoorwifeor kya...
4066,ham,baylor enron case study cindy yes i shall co a...
4067,ham,boring as compared to tp
4068,ham,hellogorgeous hows u my fone was on charge lst...


Запишем в файл

In [506]:
df_predicted.to_csv('result.csv', index=False)